
# Desafio: Acompanhamento de Ações na Bolsa de Valores B3

Uma empresa que gerencia investimentos na bolsa de valores B3 precisa desenvolver uma solução para monitorar o desempenho de ações ao longo de um período específico (ex.: semanal, mensal, trimestral). O objetivo é criar um sistema que permita a análise de dados históricos, comparando a performance de diferentes ativos, além de gerar relatórios e alertas para identificar oportunidades de compra e venda. O projeto deve incluir:

- Coleta de Dados: Captura de dados de ações listadas na B3 (valores de abertura, fechamento, volume, etc.) através de APIs ou fontes de dados públicas.

- Armazenamento: Organização dos dados históricos em uma base eficiente, possibilitando consultas rápidas.

- Análise: Desenvolvimento de modelos analíticos para identificar tendências e padrões, bem como cálculo de indicadores financeiros como P/L, dividend yield, e volatilidade.

- Visualização: Criação de dashboards e relatórios para monitorar os resultados e gerar insights sobre a performance das ações.

- Alertas: Definição de critérios de alerta para notificar a equipe sobre variações anormais ou oportunidades de mercado.



## Widgets

In [0]:
dbutils.widgets.text("acoes", "")
dbutils.widgets.text("data_inicial", "2023-01-01")
# dbutils.widgets.multiselect("media_movel","30",["10","30","60","90","120"])

In [0]:
filter_acoes = dbutils.widgets.get("acoes")
# filter_media_movel = dbutils.widgets.get("media_movel")
# filter_media_movel = ["10","30","60","90","120"]


## Install

In [0]:
!pip install yfinance 

  Using cached yfinance-0.2.45-py2.py3-none-any.whl (100 kB)
  Using cached lxml-5.3.0-cp39-cp39-manylinux_2_28_x86_64.whl (5.0 MB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
  Using cached frozendict-2.4.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (116 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached peewee-3.17.7.tar.gz (939 kB)
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - error
  ERROR: Command errored out with exit status 1:
   command: /local_disk0/.ephemeral_nfs/envs/pythonEnv-3ea32754-26c1-4751-9567-6041da7bf5de/bin/python /local_disk0/.ephemeral_nfs/envs/pythonEnv-3ea32754-26c1-4751-9567-6041da7bf5de/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py get_requires_for_build_wheel /tmp/tmpke33zcbn
       cwd: /tmp/pip-install-

In [0]:
!pip install pandas 

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3ea32754-26c1-4751-9567-6041da7bf5de/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install numpy 

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3ea32754-26c1-4751-9567-6041da7bf5de/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install matplotlib

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-3ea32754-26c1-4751-9567-6041da7bf5de/bin/python -m pip install --upgrade pip' command.


## Imports

In [0]:
import pandas as pd
import yfinance as yf

## Sets

In [0]:
data_inicial = '2019-01-01'
data_final = '2024-12-31'


# filter_acoes.split(',')

acoes = filter_acoes.split(',')

## Get 

In [0]:
df_acoes = pd.DataFrame(columns=['Open','High','Low','Close','Adj Close','Volume'])


for acao in acoes:
    # print(acao)
    tabela_cotacoes = yf.download(acao, start=data_inicial, end=data_final)
    
    tabela_cotacoes['codigo'] = acao
    tabela_cotacoes = tabela_cotacoes.reset_index()

    for media_movel in ["10","30","60","90","120"]:
        print(media_movel)
        tabela_cotacoes[f'media_movel_{media_movel}d'] = tabela_cotacoes['Close'].rolling(int(media_movel)).mean()

    # print(tabela_cotacoes.columns)
    df_acoes = pd.concat([tabela_cotacoes,df_acoes],axis=0)

    # print(tabela_cotacoes.info)

[*********************100%%**********************]  1 of 1 completed
10
30
60
90
120
[*********************100%%**********************]  1 of 1 completed
10
30
60
90
120
[*********************100%%**********************]  1 of 1 completed
10
30
60
90
120
[*********************100%%**********************]  1 of 1 completed
10
30
60
90
120


In [0]:
spark.createDataFrame(df_acoes).createOrReplaceTempView('acoes_analise')

In [0]:
colunas = 'Date::date,codigo,round(open,2) as abertura, round(close,2) as Fechamento, round(open,2) - round(close,2) as diff'

for media_movel in ["10","30","60","90","120"]:
        colunas = colunas + f',round(media_movel_{media_movel}d,2) as media_movel_{media_movel}d'

print(colunas)

display(spark.sql(f'select {colunas} from  acoes_analise ORDER BY Date::date'))

Date::date,codigo,round(open,2) as abertura, round(close,2) as Fechamento, round(open,2) - round(close,2) as diff,round(media_movel_10d,2) as media_movel_10d,round(media_movel_30d,2) as media_movel_30d,round(media_movel_60d,2) as media_movel_60d,round(media_movel_90d,2) as media_movel_90d,round(media_movel_120d,2) as media_movel_120d


Date,codigo,abertura,Fechamento,diff,media_movel_10d,media_movel_30d,media_movel_60d,media_movel_90d,media_movel_120d
2019-01-02,RAPT4.SA,9.14,9.46,-0.3200000000000003,null,null,null,null,null
2019-01-02,WEGE3.SA,8.81,8.99,-0.17999999999999972,null,null,null,null,null
2019-01-02,TUPY3.SA,20.33,20.51,-0.18000000000000327,null,null,null,null,null
2019-01-02,POMO4.SA,4.07,4.24,-0.16999999999999993,null,null,null,null,null
2019-01-03,RAPT4.SA,9.44,9.46,-0.02000000000000135,null,null,null,null,null
2019-01-03,WEGE3.SA,9.0,9.19,-0.1899999999999995,null,null,null,null,null
2019-01-03,TUPY3.SA,20.51,20.31,0.20000000000000284,null,null,null,null,null
2019-01-03,POMO4.SA,4.24,4.25,-0.009999999999999787,null,null,null,null,null
2019-01-04,RAPT4.SA,9.5,9.5,0.0,null,null,null,null,null
2019-01-04,WEGE3.SA,9.2,9.3,-0.10000000000000142,null,null,null,null,null


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

Select Date::date,codigo,open as abertura, close as Fechamento,  round(open,2) - round(close,2) as diff  from acoes_analise
order by 1

Date,codigo,abertura,Fechamento,diff
2019-01-02,RAPT4.SA,9.140000343322754,9.460000038146973,-0.3200000000000003
2019-01-02,WEGE3.SA,8.8100004196167,8.989999771118164,-0.17999999999999972
2019-01-02,TUPY3.SA,20.329999923706055,20.510000228881836,-0.18000000000000327
2019-01-02,POMO4.SA,4.070000171661377,4.239999771118164,-0.16999999999999993
2019-01-03,RAPT4.SA,9.4399995803833,9.460000038146973,-0.02000000000000135
2019-01-03,WEGE3.SA,9.0,9.194999694824219,-0.1899999999999995
2019-01-03,TUPY3.SA,20.510000228881836,20.309999465942383,0.20000000000000284
2019-01-03,POMO4.SA,4.239999771118164,4.25,-0.009999999999999787
2019-01-04,RAPT4.SA,9.5,9.5,0.0
2019-01-04,WEGE3.SA,9.199999809265137,9.300000190734863,-0.10000000000000142


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

Select Date::date, month(Date::date) as Mes ,codigo,round(open,2) as abertura, round(close,2) as Fechamento  from acoes_analise
WHERE year(Date::date) = year(current_date())
order by 1

Date,Mes,codigo,abertura,Fechamento
2024-01-02,1,RAPT4.SA,12.8,12.61
2024-01-02,1,WEGE3.SA,36.91,36.57
2024-01-02,1,TUPY3.SA,28.76,27.47
2024-01-02,1,POMO4.SA,6.99,7.32
2024-01-03,1,RAPT4.SA,12.66,13.11
2024-01-03,1,WEGE3.SA,36.38,36.85
2024-01-03,1,TUPY3.SA,27.38,26.94
2024-01-03,1,POMO4.SA,7.28,7.45
2024-01-04,1,RAPT4.SA,13.2,12.69
2024-01-04,1,WEGE3.SA,36.7,36.36


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
Select
  round(sum(close)  - sum(open) ,2) as Diff
from
  acoes_analise
WHERE
  Date :: date = (
    select
      max(Date :: date)
    from
      acoes_analise
  )

Diff
-2.32


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
with datas as (
Select
  min(Date::date) as primeira_cotacao_ano,
  max(Date::date) as ultima_cotacao_ano
from
  acoes_analise
WHERE
  YEAR(Date::date) = year(current_date())),
pega_valores as  
(select 
  case when acoes.date::date = datas.primeira_cotacao_ano then acoes.open else 0 end as abertura,
  case when acoes.date::date = datas.ultima_cotacao_ano then acoes.open else 0 end as fechamento
from  acoes_analise acoes
inner join datas datas on datas.primeira_cotacao_ano = acoes.date::date or datas.ultima_cotacao_ano = acoes.date::date 
)
select sum(abertura) as abertura,sum(fechamento) as fechamento,sum(fechamento) -sum(abertura) as lucro_ano  from pega_valores

abertura,fechamento,lucro_ano
85.46000003814697,100.96000003814697,15.5


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:

df_dividends = pd.DataFrame(columns=['Open','High','Low','Close','Adj Close','Volume'])


for acao in acoes:
    # print(acao)
    # tabela_dividends = yf.download(acao, start=data_inicial, end=data_final)

    tabela_dividends = yf.Ticker(acao).history(period="max")

    tabela_dividends['codigo'] = acao
    tabela_dividends = tabela_dividends.reset_index()

    # print(tabela_cotacoes.columns)
    df_dividends = pd.concat([df_dividends,tabela_dividends],axis=0)

    # print(tabela_cotacoes.info)




In [0]:
spark.createDataFrame(df_dividends.reset_index()).createOrReplaceTempView('dividendos')

In [0]:
%sql


select sum(dividends) from dividendos where year(Date::date) = year(current_date()) 
group by all

sum(dividends)
1.6509030000000002


Databricks visualization. Run in Databricks to view.

In [0]:
%sql


select sum(dividends) from dividendos where year(Date::date) = year(current_date())-1 
group by all

sum(dividends)
2.992345


Databricks visualization. Run in Databricks to view.

In [0]:
%sql


select sum(dividends) from dividendos where date::date between date_trunc('YEAR',date_add(current_date(),-365))::date and  date_add(current_date()-1, -366)
group by all

sum(dividends)
2.214758


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select
  sum(dividends),
  month(date :: date),
  year(date :: date),
  concat(month(date :: date),'/',year(date :: date)) as ref
from
  dividendos
where
  (date::date between date_trunc('YEAR', date_add(current_date(), -365)) :: date  and date_add(current_date() -1, -366) or 
  year(date::date) =   year(current_date()))

  and dividends > 0 

group by
  all
order by
  3,
  2

sum(dividends),month(date),year(date),ref
0.226762,1,2023,1/2023
0.226304,2,2023,2/2023
0.129902,3,2023,3/2023
0.741451,5,2023,5/2023
0.058294,6,2023,6/2023
0.42593499999999995,7,2023,7/2023
0.322777,9,2023,9/2023
0.083333,10,2023,10/2023
0.255427,1,2024,1/2024
0.297943,2,2024,2/2024


Databricks visualization. Run in Databricks to view.

In [0]:
df_teste = spark.sql('select *  from dividendos where date::date =  (select max(date::date) from dividendos)')